<h1 align=center><font size=5>Customer segmentation
  
In this challenge, we analyze the content of the [E-commerce database](https://www.kaggle.com/carrie1/ecommerce-data) that contains purchases made by about 4000 customers over a period of one year (from 2010/12/01 to 2011/12/09).
  
We then try to segment the customers into several categories, and try to classify new customers into those categories.

# 1) Data preprocessing

## 1.1) Discover kaggle API

**Task:** Download the data set, and read the description [here](https://archive.ics.uci.edu/ml/datasets/Online%20Retail) to understand the features.<br>
In order to download the dataset, we will be using Kaggle API. You can find a tutorial on how to do that here :
<a href="https://medium.com/@yvettewu.dw/tutorial-kaggle-api-google-colaboratory-1a054a382de0">Tutorial on Medium</a>


In [ ]:
!pip install kaggle

In [ ]:
# user kaggle name and kaggle key
import os
os.environ['KAGGLE_KEY'] = '5a04fa4739718910b7635c87950a0da9'   ### go to kaggle and get it 
os.environ['KAGGLE_USERNAME'] = 'niangmohamed'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1.2) Download data
Download data, using kaggle API, unzip it and delete original zip file. Look inside the file using !head

In [ ]:
!kaggle datasets list -s E-Commerce

In [ ]:
!kaggle datasets download carrie1/ecommerce-data 

from zipfile import ZipFile 
file_name = "/content/ecommerce-data.zip"
with ZipFile(file_name, 'r') as zip:                      # 'r' -> READ mode 
    zip.printdir() # print all the contents 
    zip.extractall()

del zip

!head -5 /content/data.csv

 69% 5.00M/7.20M [00:00<00:00, 47.0MB/s]
100% 7.20M/7.20M [00:00<00:00, 46.2MB/s]
File Name                                             Modified             Size
data.csv                                       2019-09-20 22:35:12     45580638
InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom


##1.3)  Read data + timestamp

**Task:** Read the data into a pandas dataframe `df`, investigate shape and head. Investigate the types of the features, and use `pd.to_datetime` to transform the timestamps features to the correct format

In [ ]:
import pandas as pd

# Read the data into a pandas dataframe `df`
df = pd.read_csv('/content/data.csv', encoding='latin', dtype={'CustomerID': str}) 

# timestamp
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

print(df.shape)
df.head()

(541909, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


In [ ]:
import numpy as np
from datetime import datetime

datetime.timestamp(np.array(df)[19][4])

1291192440.0

In [ ]:
df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID             object
Country                object
dtype: object

##1.4) Missing values

**Task:** Find the percentage of missing values for each features. Create a new dataframe `df_dropna` by removing observations without CustomerID. 

In [ ]:
# observations containing NaN
df.loc[df.isnull().sum(axis=1) > 0].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,2010-12-01 11:52:00,0.00,NaN,United Kingdom
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1445,536544,21786,POLKADOT RAIN HAT,4,2010-12-01 14:32:00,0.85,NaN,United Kingdom
1446,536544,21787,RAIN PONCHO RETROSPOT,2,2010-12-01 14:32:00,1.66,NaN,United Kingdom


In [ ]:
# missing percentages
df.isnull().sum(axis=0) / df.shape[0]

InvoiceNo      0.000000
StockCode      0.000000
Description    0.002683
Quantity       0.000000
InvoiceDate    0.000000
UnitPrice      0.000000
CustomerID     0.249267
Country        0.000000
dtype: float64

In [ ]:
# if we drop all entries with unknown customer ID, missing values in Description also disappear
df_dropna = df.dropna(axis = 0, subset = ['CustomerID', 'Description'])

# missing percentages
df_dropna.isnull().sum(axis=0) / df_dropna.shape[0]

InvoiceNo      0.0
StockCode      0.0
Description    0.0
Quantity       0.0
InvoiceDate    0.0
UnitPrice      0.0
CustomerID     0.0
Country        0.0
dtype: float64

## 1.5) Duplicates

**Task:** Is there any duplicate in the data i.e. 2 identical observations? If it's the case, remove the duplicates (keep one for each distinct obervation of course) and save to `df_dropna`.

In [ ]:
# some entries having duplicates
df_dropna.loc[df_dropna.duplicated(keep=False)].head()

In [ ]:
# duplicate example
df_dropna.loc[(df_dropna != df_dropna.loc[485]).sum(axis=1) == 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
485,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,2010-12-01 11:45:00,4.95,17908,United Kingdom
539,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,2010-12-01 11:45:00,4.95,17908,United Kingdom


In [ ]:
# drop duplicates
df_dropna.drop_duplicates(inplace=True)

# check
print("number of duplicates is ", sum(df_dropna.duplicated()))

number of duplicates is  0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
len(df_dropna)

401604

## 1.6) Save data
Save preprocessed data to the file `data_dropna.csv` and the original data to `data.csv`

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
relative_path = '/content/drive/My Drive/Customer Segmentation/Data/'
df_dropna.to_csv(relative_path + 'data_dropna.csv') 
df.to_csv(relative_path + 'data.csv') 